# Scrapeo wikipedia para sacar info de los artistas


In [1]:
import time

import warnings
warnings.filterwarnings('ignore')

from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup as bs

from selenium import webdriver

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import threading

from concurrent.futures import ThreadPoolExecutor
from selenium.common.exceptions import NoSuchElementException

import pandas as pd



In [2]:
PATH = 'driver/chromedriver.exe'

driver = webdriver.Chrome(PATH) 

# Scrap de wikipedia para un solo artista

In [3]:
url = 'https://es.wikipedia.org/wiki/Madonna'
driver.get(url)

In [4]:
driver.find_elements(By.CSS_SELECTOR, '#mw-content-text > div.mw-parser-output > table.infobox.biography.vcard')[0].text.split('\n')[4:8]
driver.quit()

# Creo funcion para sacar datos especificos de artistas

In [5]:
def info_artistas(artista, PATH):
    
    driver = webdriver.Chrome(PATH) 
    driver.get(f'https://es.wikipedia.org/wiki/{artista}')

    
    nacimiento_formacion = None
    nacionalidad_origen = None

    try:
        info = driver.find_elements(By.CSS_SELECTOR, 'table.infobox')[0].text.split('\n')
        for data in info: 
            if 'Nacimiento' in data or 'Formación' in data:
                nacimiento_formacion = data
            elif 'Nacionalidad' in data or 'Origen' in data:
                nacionalidad_origen = data.split(":")[-1].strip()
    except NoSuchElementException:
        print(f'No se encontró la infobox para el artista: {artista}')
    except Exception as e:
        print(f'Ocurrió un error al obtener la información del artista: {artista}. Error: {e}')
    
    driver.quit()

    return nacimiento_formacion, nacionalidad_origen



In [6]:
def fetch_info(artista): 
    return info_artistas(artista, PATH)

# Creo la tabla Artistas


In [7]:
artistas = pd.read_csv('../database/csv_limpios/albums.csv')

In [8]:
artistas = artistas.drop(columns= ['Year', 'Ranking','Album','Worldwide Sales','CDs','Tracks'])


In [9]:
lista_artistas = artistas['Artist'].tolist()


In [10]:
with ThreadPoolExecutor(max_workers=8) as executor: 
    #Map the `fetch_info` function to each artist
    results = list(executor.map(fetch_info, lista_artistas))


Ocurrió un error al obtener la información del artista: Eric Clapton. Error: list index out of range


In [11]:
artistas['Info'] = results

In [12]:
#Separo la columna info en dos columnas
artistas['Nacimiento_formacion'] = artistas['Info'].apply(lambda x: x[0])
artistas['Nacionalidad_origen'] = artistas['Info'].apply(lambda x: x[1])

In [13]:
artistas.drop(columns='Info')

,Artist,Genre,Nacimiento_formacion,Nacionalidad_origen
0,Madonna,Pop,Nacimiento 16 de agosto de 1958 (65 años),Nacionalidad Estadounidense
1,New_Kids_on_the_Block,Pop,None,"Origen Boston, Estados Unidos"
2,Garth Brooks,Country,Nacimiento 7 de febrero de 1962 (61 años),Nacionalidad Estadounidense
3,Metallica,Rock,None,"Origen Los Ángeles, California,"
4,Nirvana_(banda),Rock,None,"Origen Aberdeen, Washington,"
...,...,...,...,...
91,BTS,Pop,None,"Origen Seúl, Corea del Sur"
92,Pop Smoke,Hip Hop,Nacimiento 20 de julio de 1999,Nacionalidad Estadounidense
93,Adele,Pop,Nacimiento 5 de mayo de 1988 (35 años),Nacionalidad Británica
94,Olivia Rodrigo,Pop,Nacimiento 20 de febrero de 2003 (20 años),Nacionalidad Estadounidense


# Export la tabla artistas


In [14]:
artistas.to_csv('../database/csv_limpios/artistas.csv', index=False)